# Distributed Representation for Word Embeddings

In this programming assignment, we will experiment with distributed representations of words. We'll also see how such an embedding can be constructed by applying principal component analysis to a suitably transformed matrix of word co-occurrence probabilities. For computational reasons, we'll use the moderately sized **Brown corpus of present-day American English** for this.

## 1. Accessing the Brown corpus

The *Brown corpus* is available as part of the Python Natural Language Toolkit (`nltk`).

In [1]:
import numpy as np
import pickle
import nltk
nltk.download('brown')
nltk.download('stopwords')
from nltk.corpus import brown, stopwords
from scipy.cluster.vq import kmeans2
from sklearn.decomposition import PCA

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\jrsav\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jrsav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The corpus consists of 500 samples of text drawn from a wide range of sources. When these are concatenated, they form a very long stream of over a million words, which is available as `brown.words()`. Let's look at the first 50 words.

In [2]:
for i in range(20):
    print (brown.words()[i],)

The
Fulton
County
Grand
Jury
said
Friday
an
investigation
of
Atlanta's
recent
primary
election
produced
``
no
evidence
''
that


Before doing anything else, let's remove stopwords and punctuation and make everything lowercase. The resulting sequence will be stored in `my_word_stream`.

In [3]:
my_stopwords = set(stopwords.words('english'))
word_stream = [str(w).lower() for w in brown.words() if w.lower() not in my_stopwords]
my_word_stream = [w for w in word_stream if (len(w) > 1 and w.isalnum())]

Here are the initial 20 words in `my_word_stream`.

In [4]:
my_word_stream[:20]

['fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'investigation',
 'recent',
 'primary',
 'election',
 'produced',
 'evidence',
 'irregularities',
 'took',
 'place',
 'jury',
 'said',
 'presentments',
 'city',
 'executive']

## 2. Computing co-occurrence probabilities

**Task P1**: Complete the following code to get a list of words and their counts. Report how many times does the word "evidence" and "investigation" appears in the corpus.

In [5]:
N = len(my_word_stream)
# print(N)
words = []
totals = {}

## STUDENT: Your code here
# words: a python list of unique words in the document my_word_stream as the vocabulary
# totals: a python dictionary, where each word is a key, and the corresponding value
#         is the number of times this word appears in the document my_word_stream

words = my_word_stream

for word in words:
    if word in totals:
        totals[word] += 1
    else:
        totals[word] = 1
    
    
## STUDENT CODE ENDS

In [6]:
## STUDENT: Report how many times does the word "evidence" and "investigation" appears in the corpus.
print('Word "',words[11],'" appears ',totals[words[11]], ' times')
print('Word "',words[6],'" appears ',totals[words[6]], ' times')

Word " evidence " appears  204  times
Word " investigation " appears  51  times


** Task P2**: Decide on the vocabulary. There are two potentially distinct vocabularies: the words for which we will obtain embeddings (`vocab_words`) and the words we will consider when looking at context information (`context_words`). We will take the former to be all words that occur at least 20 times, and the latter to be all words that occur at least 100 times. We will stick to these choices for this assignment, but feel free to play around with them and find something better.

How large are these two word lists? Note down these numbers.

In [7]:
## STUDENT: Your code here

vocab_words = [] # a list of words whose occurances (totals) are > 19
context_words = [] # a list of words whose occurances (totals) are > 99

wordList = totals.items()
for word in wordList:
    if word[1] > 99:
        context_words.append(word[0])
    if word[1] > 19:
        vocab_words.append(word[0])
        
## STUDENT CODE ENDS
print ('Number of vocabulary words ',len(vocab_words), ';')
print ('Number of context words ',len(context_words), ';' )
#print(context_words)

Number of vocabulary words  4720 ;
Number of context words  918 ;


**Task P3**: Get co-occurrence counts. These are defined as follows, for a small constant `window_size=2`.

* Let `w0` be any word in `vocab_words` and `w` any word in `context_words`.
* Each time `w0` occurs in the corpus, look at the window of `window_size` words before and after it. If `w` appears in this window, we say it appears in the context of (this particular occurrence of) `w0`.
* Define `counts[w0][w]` as the total number of times `w` occurs in the context of `w0`.

Complete the function `get_counts`, which computes the `counts` array and returns it as a dictionary (of dictionaries). Find how many times the word "fact" appears in the context of ”evidence" with window_size=2.

In [8]:
def get_counts(window_size=2):
    ## Input:
    #  window_size: for each word w0, its context includes window_size words before and after it. 
    #  For instance, if window_size = 2, it means we look at w1 w2 w0 w3 w4, where  w1, w2, w3, w4 are 
    #  context woreds
    ## Output:
    #  counts: a python dictionary (of dictionaries) where counts[w0][w] indicate the number of times the word w appears 
    #  in the context of w0 (Note: counts[w0] is also a python dictionary)
    
    counts = {}
    for w0 in vocab_words:
        counts[w0] = {} 
        
    ## STUDENT: Your code here
    
    #set each vocab word to have counter for all possible context words
    for w0 in vocab_words:
        for word in context_words:
            counts[w0][word]= 0
        
    vocab_list = list(counts.keys())
    words = my_word_stream
    index_list = []
    window_list = []
    vocab_index = 0
    indexMin = 0
    indexMax = 0
    word_of_interest = ""
    
    #iterate through the vocab words
    for i in range(len(vocab_list)):
        
        #get the current vocab_word
        word_of_interest = vocab_list[i]
        
        #find the index of every occurence of the current vocab word in the corpus
        index_list = [i for i,word in enumerate(words) if word == word_of_interest]
        
        #print(index_list)
        #go to every occurence of the vocab word in the corpus and check the window size
        for i in index_list:

            indexMin = i - window_size
            indexMax = i + window_size + 1

            if indexMin < 0:
                indexMin = 0

            if indexMax > len(words) - 1:
                indexMax = len(words)

            window_list = words[indexMin:indexMax]
            window_list.remove(word_of_interest)
            #print(window_list)

            for word in context_words:
                if word in window_list:
                    counts[word_of_interest][word] += 1
        
    ## End of codes
    return counts

In [9]:
## STUDENT: Report how many times the word "fact" appears in the context of ”evidence".
counts = get_counts(window_size=2)
print (counts['evidence']['fact'])

4


Define `probs[w0][]` to be the distribution over the context of `w0`, that is:
* `probs[w0][w] = counts[w0][w] / (sum of all counts[w0][])`

**Task P4**: Finish the function `get_co_occurrence_dictionary` that computes `probs`. Find the probability that the word "fact" appears in the context of ”evidence".

In [10]:
def get_co_occurrence_dictionary(counts):
    ## Input:
    #  counts: a python dictionary (of dictionaries) where counts[w0][w] indicate the number of times the word w appears 
    #  in the context of w0 (Note: counts[w0] is also a python dictionary)
    ## Output:
    #  probs: a python dictionary (of dictionaries) where probs[w0][w] indicate the probability that word w appears 
    #  in the context of word w0
    
    probs = {}
    
    ## STUDENT: Your code here
    #initialize probability dictionary to have each vocab word have a probability of every context word be initially zero
    for w0 in vocab_words:
        probs[w0] = {} 
        for word in context_words:
            probs[w0][word]= 0
        
    for w0 in list(probs.keys()):
        for w in list(probs[w0].keys()):
            probs[w0][w] = counts[w0][w]/sum( list( counts[w0].values() ) )
            
    ## End of codes
    return probs

In [11]:
## STUDENT: Report how many times the word "fact" appears in the context of ”evidence".
probs = get_co_occurrence_dictionary(counts)
print (probs['evidence']['fact'])

0.010723860589812333


The final piece of information we need is the frequency of different context words. The function below, `get_context_word_distribution`, takes `counts` as input and returns (again, in dictionary form) the array:

* `context_frequency[w]` = sum of all `counts[][w]` / sum of all `counts[][]` 

In [12]:
def get_context_word_distribution(counts):
    counts_context = {}
    sum_context = 0
    context_frequency = {}
    for w in context_words:
        counts_context[w] = 0
    for w0 in counts.keys():
        for w in counts[w0].keys():
            counts_context[w] = counts_context[w] + counts[w0][w]
            sum_context = sum_context + counts[w0][w]
    for w in context_words:
        context_frequency[w] = float(counts_context[w])/float(sum_context)
    return context_frequency

## 3. The embedding

**Task P5**: Based on the various pieces of information above, we compute the **pointwise mutual information matrix**:
* `PMI[i,j] = MAX(0, log probs[ith vocab word][jth context word] - log context_frequency[jth context word])`

Complete the code to compute PMI for every word i and context word j. Report the output of the code.

In [13]:
print ("Computing counts and distributions")
counts = get_counts(2)
probs = get_co_occurrence_dictionary(counts)
context_frequency = get_context_word_distribution(counts)
#
print ("Computing pointwise mutual information")
n_vocab = len(vocab_words)
n_context = len(context_words)
pmi = np.zeros((n_vocab, n_context))
for i in range(0, n_vocab):
    w0 = vocab_words[i]
    for w in probs[w0].keys():
        j = context_words.index(w)
        ## STUDENT: Your code here
        pmi[i,j] = max( 0, np.log(probs[w0][context_words[j]] ) - np.log(context_frequency[context_words[j]]) )
        ## Student end of code

Computing counts and distributions
Computing pointwise mutual information


<ipython-input-13-c9b5bb3dc9ab>:15: RuntimeWarning: divide by zero encountered in log
  pmi[i,j] = max( 0, np.log(probs[w0][context_words[j]] ) - np.log(context_frequency[context_words[j]]) )


In [14]:
# STUDENT: report the following number
print(pmi[vocab_words.index('evidence'),context_words.index('fact')])

1.6800791816244818


The embedding of any word can then be taken as the corresponding row of this matrix. However, to reduce the dimension, we will apply principal component analysis (PCA).

See this nice tutorial on PCA: https://www.youtube.com/watch?v=fkf4IBRSeEc

Now reduce the dimension of the PMI vectors using principal component analysis. Here we bring it down to 100 dimensions, and then normalize the vectors to unit length.

In [15]:
pca = PCA(n_components=100)
vecs = pca.fit_transform(pmi)
for i in range(0,n_vocab):
    vecs[i] = vecs[i]/np.linalg.norm(vecs[i])

It is useful to save this embedding so that it doesn't need to be computed every time.

In [16]:
fd = open("embedding.pickle", "wb")
pickle.dump(vocab_words, fd)
pickle.dump(context_words, fd)
pickle.dump(vecs, fd)
fd.close()

## 4. Experimenting with the embedding

We can get some insight into the embedding by looking at some intersting use cases.

** Task P6**: Implement the following function that finds the nearest neighbor of a given word in the embedded space. Note down the answers to the following queries. 

In [17]:
def word_NN(w,vecs,vocab_words,context_words):
    ## Input:
    #  w: word w
    #  vecs: the embedding of words, as computed above
    #  vocab_words: vocabulary words, as computed in Task P2
    #  context_words: context words, as computed in Task P2
    ## Output:
    #  the nearest neighbor (word) to word w
    if not(w in vocab_words):
        print ("Unknown word")
        return
    
    ## Student: your code here
    vect = vecs[vocab_words.index(w)]
    neighbor = 0
    curr_dist = np.linalg.norm(vect - vecs[0])
    for i in range(1, len(vocab_words)):
        dist = np.linalg.norm(vect - vecs[i])
        if(dist < curr_dist) and (dist > 0.0):
            neighbor = i
            curr_dist = dist
    
    return vocab_words[neighbor]
    ## Student: code ends

In [18]:
word_NN('world',vecs,vocab_words,context_words)

'nations'

In [19]:
word_NN('learning',vecs,vocab_words,context_words)

'freedom'

In [20]:
word_NN('technology',vecs,vocab_words,context_words)

'studies'

In [21]:
word_NN('man',vecs,vocab_words,context_words)

'woman'

** Task P7**: Implement the function that aims to solve the analogy problem:
A is to B as C is to ?
For example, A=King, B=Queen, C=man, and the answer for ? should be ideally woman (you will see that this may not be  the case using the distributed representation).

Finds the K-nearest neighbor of a given word in the embedded space. Note: instead of outputing only the nearest neighbor, you should find the K=10 nearest neighbors and see whether there is one in the list that makes sense. You should also exclude the words C in the output list.

Also report another set A, B, C and the corresponding answer output by your problem. See if it makes sense to you.

In [59]:
def find_analogy(A,B,C,vecs,vocab_words,context_words):
    ## Input:
    #  A, B, C: words A, B, C
    #  vecs: the embedding of words, as computed above
    #  vocab_words: vocabulary words, as computed in Task P2
    #  context_words: context words, as computed in Task P2
    ## Output:
    #  the word that solves the analogy problem
    ## STUDENT: Your code 
    import operator
    query_word = vecs[vocab_words.index(C)] + vecs[vocab_words.index(B)] - vecs[vocab_words.index(A)]
    distances = []
    for index in range(1, len(vocab_words)):
        dist = np.linalg.norm(query_word - vecs[index])
        
        if(vocab_words[index] != A) and (vocab_words[index] != B) and (vocab_words[index] != C):
            distances.append([index, dist, vocab_words[index]])
            
    distances.sort(key=operator.itemgetter(1))
    
    neighbors = []
    
    for i in range(10):
        neighbors.append(distances[i][0])
    
    output_vals = []
    for i in range(len(neighbors)):
        output_vals.append(vocab_words[neighbors[i]])
    
    return output_vals
    ## STUDENT: your code ends


In [60]:
find_analogy('king','queen','man',vecs,vocab_words,context_words)

['soul',
 'kate',
 'spoke',
 'dying',
 'wonder',
 'loved',
 'creatures',
 'blame',
 'gentle',
 'old']

In [61]:
find_analogy('soil','grass','sun',vecs,vocab_words,context_words)

['dark',
 'closed',
 'blue',
 'rain',
 'saw',
 'around',
 'suddenly',
 'hung',
 'stood',
 'rose']

In [66]:
find_analogy('mayor','city','president',vecs,vocab_words,context_words)

['local',
 'report',
 'members',
 'state',
 'department',
 'service',
 'government',
 'board',
 'today',
 'kennedy']